In [2]:
from nba_api.stats.static import players, teams
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscorescoringv2
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import boxscoretraditionalv2
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import time as time
from time import sleep
from IPython.display import clear_output
import sqlite3

In [3]:
pd.options.display.max_columns=60

In [4]:
def season_string(season):
    return str(season) + '-' + str(season+1)[-2:]

In [5]:
def add_basic_boxscores(conn, start_season, end_season, if_exists='append'):
    """This function pulls basic team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_basic_boxscores in the sqlite db"""
    
    table_name = 'team_basic_boxscores'
    
    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)
        conn.execute('VACUUM')
        
    conn.execute("""CREATE TABLE IF NOT EXISTS {} (SEASON TEXT, TEAM_ID INTEGER, TEAM_ABBREVIATION TEXT, 
        TEAM_NAME TEXT, GAME_ID TEXT, GAME_DATE DATE, MATCHUP TEXT, WL TEXT, MIN INTEGER, FGM INTEGER, FGA INTEGER, 
        FG_PCT FLOAT, FG3M INTEGER, FG3A  INTEGER, FG3_PCT FLOAT, FTM INTEGER, FTA INTEGER, FT_PCT FLOAT, OREB INTEGER,
        DREB INTEGER, REB INTEGER, AST INTEGER, STL INTEGER, BLK INTEGER, TOV INTEGER, PF INTEGER, PTS INTEGER, 
        PLUS_MINUS INTEGER)""".format(table_name))    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)

        for season_type in ['Regular Season', 'PlayIn',  'Playoffs']:
            boxscores = leaguegamelog.LeagueGameLog(season=season_str, season_type_all_star=season_type).get_data_frames()[0]
            season_boxscores.append(boxscores)
            sleep(2)
        season_df = pd.concat(season_boxscores)
        season_df['SEASON'] = season_str
        season_df.drop(columns = ['SEASON_ID', 'VIDEO_AVAILABLE'], inplace=True)
        
        season_df.to_sql(table_name, conn, if_exists='append', index=False)
        
        sleep(3)
        
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return None

In [6]:
def add_advanced_boxscores(conn, start_season, end_season, if_exists='append'):
    """
    This function pulls advanced team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_advanced_boxscores in the sqlite db
    
    Note: Because of timeout errors and that we have to pull each game's individually, each season takes 1-2 hours.
    If some games were not pulled in certain seasons, you can use the update functions to gather those individual games.
    """
    
    table_name = 'team_advanced_boxscores'
    game_ids_not_added = []
    
    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)
        conn.execute('VACUUM')
    
    conn.execute('''CREATE TABLE IF NOT EXISTS {} (GAME_ID TEXT, TEAM_ID INTEGER, TEAM_NAME TEXT, 
        TEAM_ABBREVIATION TEXT, TEAM_CITY TEXT, MIN TEXT, E_OFF_RATING FLOAT, OFF_RATING FLOAT, E_DEF_RATING FLOAT, 
        DEF_RATING FLOAT, E_NET_RATING FLOAT, NET_RATING FLOAT, AST_PCT FLOAT, AST_TOV FLOAT, 
        AST_RATIO FLOAT, OREB_PCT FLOAT, DREB_PCT FLOAT, REB_PCT FLOAT, E_TM_TOV_PCT FLOAT, 
        TM_TOV_PCT FLOAT, EFG_PCT FLOAT, TS_PCT FLOAT, USG_PCT FLOAT, E_USG_PCT FLOAT, E_PACE FLOAT, 
        PACE FLOAT, PACE_PER40 FLOAT, POSS FLOAT, PIE FLOAT)'''.format(table_name))
    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)
        season_team_boxscores = []

        for season_type in ['Regular Season', 'PlayIn', 'Playoffs'] :
            logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
            game_ids = logs['GAME_ID'].unique()

            for i in range(0, len(game_ids), 100):
                print('games {} to {}'.format(i, i+100))
                for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                    try:
                        team_boxscores = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id).get_data_frames()[1]                    
                        team_boxscores.to_sql(table_name, conn, if_exists='append', index=False)
                    except:
                        game_ids_not_added.append(game_id)
                    sleep(2)
                sleep(120)
                clear_output(wait=True)

        sleep(120)
        
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return None

In [7]:
def add_scoring_boxscores(conn, start_season, end_season, if_exists='append'):
    """
    This function pulls scoring team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_scoring_boxscores in the sqlite db.
    
    Note: Because of timeout errors and that we have to pull each game's individually, each season takes 1-2 hours.
    If some games were not pulled in certain seasons, you can use the update functions to gather those individual games.
    """
    
    table_name = 'team_scoring_boxscores'
    game_ids_not_added = []

    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + table_name)
        conn.execute('VACUUM')
    
    conn.execute('''GAME_ID TEXT, TEAM_ID INTEGER, TEAM_NAME TEXT, TEAM_ABBREVIATION TEXT, TEAM_CITY TEXT,
       MIN TEXT, PCT_FGA_2PT FLOAT, PCT_FGA_3PT FLOAT, PCT_PTS_2PT FLOAT, PCT_PTS_2PT_MR FLOAT,
       PCT_PTS_3PT FLOAT, PCT_PTS_FB FLOAT, PCT_PTS_FT FLOAT, PCT_PTS_OFF_TOV FLOAT,
       PCT_PTS_PAINT FLOAT, PCT_AST_2PM FLOAT, PCT_UAST_2PM FLOAT, PCT_AST_3PM FLOAT,
       PCT_UAST_3PM FLOAT, PCT_AST_FGM FLOAT, PCT_UAST_FGM FLOAT)'''.format(table_name))
    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)
        season_team_boxscores = []

        for season_type in ['Regular Season', 'PlayIn', 'Playoffs'] :
            logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
            game_ids = logs['GAME_ID'].unique()

            for i in range(0, len(game_ids), 100):
                print('games {} to {}'.format(i, i+100))
                for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                    try:
                        scoring_boxscores = boxscorescoringv2.BoxScoretrackingV2(game_id).get_data_frames()[1]
                        tracking_boxscores.to_sql(table_name, conn, if_exists='append', index=False)
                    except:
                        game_ids_not_added.append(game_id)
                    sleep(2)
                sleep(120)
                clear_output(wait=True)

        sleep(120)
        
    cur = conn.cursor()
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(table_name, table_name))
    conn.commit()
    
    return game_ids_not_added

In [8]:
game_id = leaguegamelog.LeagueGameLog(season=2013).get_data_frames()[0]['GAME_ID'].values[0]

df = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id).get_data_frames()[0]

In [9]:
team_tracking_df = pd.DataFrame()
player_tracking_df = pd.DataFrame()


In [10]:
start_season = 2023
end_season = 2023
game_ids_not_added = []

for season in range(start_season, end_season+1):
    season_str = season_string(season)
    season_team_tracking_boxscores = []
    season_player_tracking_boxscores = []

    for season_type in ['Regular Season', 'PlayIn', 'Playoffs'] :
        logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
        game_ids = logs['GAME_ID'].unique()[:10]

        for i in range(0, len(game_ids), 100):
            print('games {} to {}'.format(i, i+100))
            for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                try:
                    tracking_boxscores = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id).get_data_frames()
                    
                    # scoring_boxscores.to_sql(table_name, conn, if_exists='append', index=False)
                    season_team_tracking_boxscores.append(tracking_boxscores[1])
                    season_player_tracking_boxscores.append(tracking_boxscores[0])
                    
                except:
                    game_ids_not_added.append(game_id)
                sleep(1)
    clear_output(wait=True)


team_tracking_df = pd.concat(season_team_tracking_boxscores)
player_tracking_df = pd.concat(season_player_tracking_boxscores)

games 0 to 100


progress: 100%|██████████| 10/10 [00:12<00:00,  1.25s/it]


games 0 to 100


progress: 100%|██████████| 6/6 [00:08<00:00,  1.37s/it]


games 0 to 100


progress: 100%|██████████| 10/10 [00:20<00:00,  2.05s/it]


In [12]:
team_tracking_df.to_csv('data/team_tracking.csv', index=False)


In [ ]:
def add_tracking_boxscores(conn, start_season, end_season, if_exists='append'):
    """
    This function pulls scoring team boxscores from the NBA_API package 
    and appends (or creates a new table if not exists) it to the table team_tracking_boxscores in the sqlite db.
    
    Note: Because of timeout errors and that we have to pull each game's individually, each season takes 1-2 hours.
    If some games were not pulled in certain seasons, you can use the update functions to gather those individual games.
    """
    
    player_table_name = 'player_tracking_boxscores'
    team_table_name = 'team_tracking_boxscores'

    game_ids_not_added = []

    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + player_table_name)
        conn.execute('VACUUM')
    
    conn.execute('''CREATE TABLE IF NOT EXISTS {} (
                GAME_ID TEXT, TEAM_ID TEXT, TEAM_ABBREVIATION TEXT, 
                TEAM_CITY TEXT, PLAYER_ID TEXT, PLAYER_NAME TEXT,
                START_POSITION TEXT, COMMENT TEXT, MIN INTEGER, SPD FLOAT, 
                DIST FLOAT, ORBC INTEGER, DRBC INTEGER, RBC INTEGER, 
                TCHS INTEGER, SAST INTEGER, FTAST INTEGER, PASS INTEGER,
                AST INTEGER, CFGM INTEGER, CFGA INTEGER, CFG_PCT FLOAT,
                UFGM INTEGER, UFGA INTEGER, UFG_PCT FLOAT, FG_PCT FLOAT, 
                DFGM INTEGER, DFGA INTEGER, DFG_PCT FLOAT)'''.format(player_table_name))
        
    if if_exists == 'replace':
        conn.execute('DROP TABLE IF EXISTS ' + team_table_name)
        conn.execute('VACUUM')
    
    conn.execute('''CREATE TABLE IF NOT EXISTS {} (
                GAME_ID TEXT, TEAM_ID TEXT, TEAM_NAME TEXT, TEAM_ABBREVIATION TEXT, 
                TEAM_CITY TEXT, MIN INTEGER, DIST FLOAT, ORBC INTEGER, DRBC INTEGER, RBC INTEGER, 
                TCHS INTEGER, SAST INTEGER, FTAST INTEGER, PASS INTEGER,
                AST INTEGER, CFGM INTEGER, CFGA INTEGER, CFG_PCT FLOAT,
                UFGM INTEGER, UFGA INTEGER, UFG_PCT FLOAT, FG_PCT FLOAT, 
                DFGM INTEGER, DFGA INTEGER, DFG_PCT FLOAT)'''.format(team_table_name))    
    
    
    for season in range(start_season, end_season+1):
        season_str = season_string(season)

        for season_type in ['Regular Season', 'PlayIn', 'Playoffs'] :
            logs = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
            game_ids = logs['GAME_ID'].unique()[:10]

            for i in range(0, len(game_ids), 100):
                print('games {} to {}'.format(i, i+100))
                for game_id in tqdm(game_ids[i:i+100], desc='progress'):
                    try:
                        tracking_boxscores = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id).get_data_frames()
                        
                        team_tracking_boxscores = tracking_boxscores[1]
                        player_tracking_boxscores = tracking_boxscores[0]
                        
                        player_tracking_boxscores.to_sql(player_table_name, conn, if_exists='append', index=False)
                        team_tracking_boxscores.to_sql(team_table_name, conn, if_exists='append', index=False)
    
                    except:
                        game_ids_not_added.append(game_id)
                    sleep(2)
                sleep(120)
                clear_output(wait=True)

        sleep(120)
        
    cur = conn.cursor()
    
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY PLAYER_ID, GAME_ID)'.format(player_table_name, player_table_name))
    cur.execute('DELETE FROM {} WHERE rowid NOT IN (SELECT min(rowid) FROM {} GROUP BY TEAM_ID, GAME_ID)'.format(team_table_name, team_table_name))
    
    conn.commit()
    
    return game_ids_not_added